In [1]:
# df
import pandas as pd
import string
import numpy as np
import time

# visualize
import matplotlib.pyplot as plt
import seaborn as sns

# nlp libraries
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# preprocessing tools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# neural net's  and pipeline library
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.callbacks import EarlyStopping


# unputting data
train=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\train.csv')
test=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\test.csv')
sample_submission=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\sample_submission.csv')


### cleaner function
def text_cleaner (text) :
    punctuation = [char for char in text if char not in string.punctuation]
    punctuation = ''.join (punctuation)
    return [word for word in punctuation.split() if word.lower() not in stopwords.words('english')]


In [2]:
%%time
transformer = CountVectorizer(analyzer=text_cleaner).fit(train['text'])

transformer_bow = transformer.transform(train['text'])

tf_idf_transformer = TfidfTransformer().fit(transformer_bow)

text_tfidf = tf_idf_transformer.transform(transformer_bow)


CPU times: total: 24.3 s
Wall time: 25.7 s


In [3]:
%%time

ann = tf.keras.models.Sequential()

early_stop = EarlyStopping(monitor='accuracy',mode='min')

ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

CPU times: total: 0 ns
Wall time: 56.5 ms


In [29]:
ann.fit(x=text_tfidf.toarray(),y=train['target'],batch_size = 331,epochs =100000,callbacks=[early_stop])       

Epoch 1/100000
23/23 [==============================] - 1s 16ms/step - loss: 0.6747 - accuracy: 0.6220
Epoch 2/100000
23/23 [==============================] - 0s 10ms/step - loss: 0.5279 - accuracy: 0.8631


In [38]:
ann_prediction = ann.predict(x=text_tfidf.toarray())
ann_prediction = (ann_prediction > 0.5)
ann_prediction

238/238 [==============================] - 1s 3ms/step


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [41]:
print (classification_report(train['target'], ann_prediction))

              precision    recall  f1-score   support

           0       0.89      0.99      0.93      4342
           1       0.98      0.83      0.90      3271

    accuracy                           0.92      7613
   macro avg       0.93      0.91      0.92      7613
weighted avg       0.93      0.92      0.92      7613



In [22]:
# data = pd.DataFrame({'id': train.id,'original_target' : train.target,'target_predicted': np.ravel(ann_prediction)})
# data['target_predicted'][data['target_predicted']==True]=int(1)
# data['target_predicted'][data['target_predicted']==False]=int(0)
# data

In [35]:
%%time
transformer_1 = CountVectorizer(analyzer=text_cleaner).fit(test['text'])

transformer_bow_1 = transformer_1.transform(test['text'])

tf_idf_transformer_1 = TfidfTransformer().fit(transformer_bow_1)

text_tfidf_1 = tf_idf_transformer_1.transform(transformer_bow_1)
